In [1]:
import MagmaPandas.volatile_solubility.models.IaconoMarziano as im
import MagmaPandas.volatile_solubility.models.Allison2022 as a
import MagmaPandas.volatile_solubility.models.shiskina as sh
sh_config = sh.shiskina_configuration
sh_config.print()
# print("\n")
# a_config = a.Allison_configuration
# a_config.print()

##### Shiskina volatile solubility #####
########################################
Settings________________________________
Fugacity model.....................ideal

Calibration range______________________
Temperature..................1423-1523°K
Pressure......................0.2-5 kbar




In [2]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.parse_io.readers as r
import pandas as pd
import elements as e
import numpy as np
import matplotlib.pyplot as plt
import VESIcal as v
config = mp.configuration
print(config)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................1
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed


/Users/thomas/miniforge3/lib/python3.9/site-packages/VESIcal/calculate_classes.py:7: UserWarning: 

  from VESIcal.models import magmasat


In [3]:

melt = mp.read_melt('./melt.csv', index_col=['name'], total_col='total')

In [6]:
config.volatile_solubility = "Allison2022"
melt.volatile_saturation_pressure(1400)

|██████████████████████████████| 29/29 [100%] in 3.4s (8.42/s)                                                          23/29 [79%] in 2s (10.5/s, eta: 1s) 


name
PI032-01-01    2455.798938
PI032-01-04    2665.571696
PI032-02-01    2464.518548
PI032-02-02    2625.414051
PI032-02-03    2433.940770
PI032-03-01    2980.914992
PI032-04-01    3210.365792
PI032-04-02    3878.538100
PI032-04-03     432.078972
PI032-05-01    3445.968541
PI041-02-01    3305.089875
PI041-02-02    2171.634669
PI041-03-01    3653.416977
PI041-03-03    3750.295344
PI041-04-02    4428.174676
PI041-04-03    4410.842640
PI041-04-04    4288.526570
PI041-04-05    4527.961509
PI041-04-06    4294.099361
PI041-04-07    4512.630147
PI041-05-02    2788.948953
PI041-05-03    2792.203792
PI041-05-04    1868.927708
PI041-05-06    3084.813844
PI041-07-01    3897.921258
PI041-07-02    2956.813065
PI041-07-06    3345.630140
PI052-01-02    1074.813409
PI052-02-01    1639.876834
dtype: float64

In [ ]:
sample = "PI032-04-01"
FeO_initial = 15
Fo_host = 0.6
Kd = 0.29
P_bar = 1e3
T_K = 1500
inclusion = melt.loc[sample,:].copy()
# inclusion["H2O"] = 5


In [ ]:
import MagmaPandas.volatile_solubility as vs
vs.calculate_saturation(inclusion, 1400)

In [ ]:
s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
s.get_composition()

In [ ]:
v.get_model_names()

In [ ]:
print(f"{'Sample': <15}{'VESIcal':<10}{'Me':<10}{'delta': <7}{'X-H2O': >}")
for idx in range(20):

    inclusion = melt.iloc[idx,:]
    s = v.Sample(inclusion[inclusion.index.intersection(v.oxides)])
    model = "IaconoMarziano"
    VC = v.calculate_saturation_pressure(s, temperature=T_K-273, model=model).result
    IM, xIM = im.calculate_saturation(inclusion, T_K, output="both")
    # A, xA = a.calculate_saturation(inclusion, T_K, output="both")
    # SH, x = sh.mixed.calculate_saturation(inclusion, output="both")
    # string = f"IM: {IM:>5.0f}, A: {A:>5.0f}"    
    print(f"{inclusion.name:<15}{VC:<10.0f}{IM:<10.0f}{(VC - IM): <7.0f}{xIM: >.2f}")
    # print(f"VESIcal: {VC:<10.0f} {string:>25}, delta: {(VC - IM): >3.0f} bar, Xfl-IM: {xim: >4.2f}, Xfl-A: {x: .2f}")

In [ ]:
model = "IaconoMarzianoWater"

p = np.arange(1,  1e4, 5e1)
fig, ax = plt.subplots(figsize=(10, 10))

plt.plot(p, [im.h2o.calculate_solubility(inclusion, P_bar=i, T_K=T_K) for i in p], label="Me", linewidth=4, alpha=0.8, linestyle="--")
plt.plot(p, [v.calculate_dissolved_volatiles(s, temperature=T_K-273.15, pressure=i, model=model).result for i in p], label="VESIcal")

plt.legend()
plt.show()

In [ ]:
melt.iloc[0, :]